In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd.functional import jacobian
from functorch import vmap, jacrev
import matplotlib.pyplot as plt

In [2]:
# ------------------------------
# Set up the device: GPU if available, otherwise CPU
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ------------------------------
# Define ODE constants and the ODE function φ on the selected device
# ------------------------------
D = torch.tensor([-9.54, -8.16, -4.26, -11.43], dtype=torch.float32, device=device)
A = torch.tensor(
    [[3.18, 2.72, 1.42, 3.81]], dtype=torch.float32, device=device
)  # shape: (4,)
b = torch.tensor([[7.81]], dtype=torch.float32, device=device)  # scalar


# def phi(y):
#     """
#     Given y (a tensor of shape (5,)), compute:
#       x = y[:4]
#       u = y[4:5]
#       m = max(0, u + dot(A, x) - b)
#     and return
#       [ -(D + A*m); m - u ] as a tensor of shape (5,)
#     """
#     x = y[:4]
#     u = y[4]
#     m = torch.clamp(u + torch.dot(A, x) - b, min=0.0)
#     top = -(D + A * m)
#     bottom = m - u
#     return torch.cat((top, bottom.unsqueeze(0)))  # resulting shape: (5,)


def createPhi(D, A, b):
    _, n = A.shape

    def phi(y):
        """
        Given y (a tensor of shape (6,)), where:
          x = y[:4] ∈ ℝ⁴,
          u = y[4:] ∈ ℝ² (treated as a column vector),
        compute:
          m = max(0, u + A @ x - b) ∈ ℝ²,
        and return:
          [ -(D + Aᵀ @ m);  m - u ] as a tensor of shape (6,).

        Note: This function assumes:
          - A is a tensor of shape (2,4)
          - D is a tensor of shape (4,)
          - b is a scalar.
        """
        # Convert x to a column vector of shape (4,1)
        x = y[:n]  # shape: (4,1)
        # Convert u to a column vector of shape (2,1)
        u = y[n:]  # shape: (2,1)

        # Compute m = max(0, u + A @ x - b)
        m = torch.clamp(u + A @ x - b, min=0.0)  # shape: (2,1)

        # Compute top = -(D + Aᵀ @ m)
        # Ensure D is used as a column vector by unsqueezing it.
        top = -(D.unsqueeze(1) + A.t() @ m)  # shape: (4,1)

        # Compute bottom = m - u
        bottom = m - u  # shape: (2,1)

        # Concatenate top and bottom along the first dimension and squeeze to get shape (6,)
        return torch.cat((top, bottom), dim=0).squeeze(1)

    return phi


phi = createPhi(D, A, b)

Using device: cuda


In [3]:
# ------------------------------
# Define the PINN model (moved to GPU)
# ------------------------------
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        # Two-layer network: input dimension 1 -> 100 -> 5
        self.fc1 = nn.Linear(1, 100)
        self.fc2 = nn.Linear(100, 5)
        self.activation = nn.Tanh()

    def forward(self, t):
        """
        Forward pass for a scalar (or batch) time input.
        We assume t is a tensor of shape (N, 1) or a scalar tensor.
        The network output is modulated as:
            ŷ(t) = (1 - exp(-t)) * NN(t)
        to enforce ŷ(0) = 0.
        """
        if t.dim() == 0:
            t = t.unsqueeze(0)
        x = self.activation(self.fc1(t))
        out = self.fc2(x)
        return (1 - torch.exp(-t)) * out


model = PINN().to(device)

In [4]:
# ------------------------------
# Set up collocation points on the device
# ------------------------------
# 100 points uniformly in [0,10]
ts = torch.linspace(0, 10, 128, dtype=torch.float32, device=device)  # shape (100,)


# ------------------------------
# Define the vectorized loss function using functorch
# ------------------------------
def compute_loss_vectorized():
    # ts: shape (N,) ; we need to work with scalar inputs, so we keep ts as a 1D tensor.
    # Evaluate the PINN on all collocation points.
    # The model expects input shape (N,1), so unsqueeze ts.
    ts_var = ts.clone().detach().requires_grad_(True)  # shape (N,)
    y_hat = model(ts_var.unsqueeze(1))  # shape: (N, 5)

    # Define a function that maps a scalar t to the model output (a vector of shape (5,))
    def model_single(t):
        # t is a scalar; model expects shape (1,1)
        return model(t.unsqueeze(0)).squeeze(0)

    # Compute the derivative dy/dt for each scalar time t using vectorized jacobian.
    # jacrev computes the Jacobian of model_single at a scalar t (output shape: (5,))
    dy_dt = torch.vmap(torch.func.jacrev(model_single))(ts_var)  # shape: (N, 5)

    # Vectorize phi over the batch dimension.
    phi_y = torch.vmap(phi)(y_hat)  # shape: (N, 5)

    # Compute the residuals at each collocation point.
    residuals = dy_dt - phi_y  # shape: (N, 5)
    # Compute the mean squared residual over the collocation points.
    loss = torch.mean(torch.sum(residuals**2, dim=1))
    return loss

In [5]:
# ------------------------------
# Training Loop using Adam (lr = 0.001) on the GPU
# ------------------------------
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 500000

print("Starting training...")
for epoch in range(1, epochs + 1):
    optimizer.zero_grad()
    loss_val = compute_loss_vectorized()
    loss_val.backward()
    optimizer.step()

    # Print loss every 100 epochs.
    if epoch % 10 == 0:
        print(f"Epoch {epoch}: Loss = {loss_val.item():.6f}")

    # Stop if loss is below the threshold.
    if loss_val.item() < 1e-4:
        print(f"Stopping training at epoch {epoch} with loss = {loss_val.item():.6f}")
        break

print("Training complete.\n")

# ------------------------------
# Evaluate the trained model at select time points
# ------------------------------
test_times = [0.0, 2.5, 5.0, 7.5, 10.0]
for t in test_times:
    t_tensor = torch.tensor(t, dtype=torch.float32, device=device, requires_grad=True)
    y_pred = model(t_tensor)
    print(f"t = {t:4.1f}, ŷ(t) = {y_pred.detach().cpu().numpy().flatten()}")

Starting training...
Epoch 10: Loss = 70.153664
Epoch 20: Loss = 67.232086
Epoch 30: Loss = 65.588097
Epoch 40: Loss = 59.980167
Epoch 50: Loss = 53.729893
Epoch 60: Loss = 50.108990
Epoch 70: Loss = 48.142227
Epoch 80: Loss = 46.178013
Epoch 90: Loss = 44.450001
Epoch 100: Loss = 42.716206
Epoch 110: Loss = 41.031857
Epoch 120: Loss = 39.452362
Epoch 130: Loss = 37.792831
Epoch 140: Loss = 36.303318
Epoch 150: Loss = 34.879406
Epoch 160: Loss = 33.441963
Epoch 170: Loss = 32.191849
Epoch 180: Loss = 31.044241
Epoch 190: Loss = 29.846729
Epoch 200: Loss = 28.811152
Epoch 210: Loss = 27.938787
Epoch 220: Loss = 27.108126
Epoch 230: Loss = 26.230343
Epoch 240: Loss = 25.477045
Epoch 250: Loss = 24.841742
Epoch 260: Loss = 24.305769
Epoch 270: Loss = 23.770065
Epoch 280: Loss = 23.145992
Epoch 290: Loss = 22.602079
Epoch 300: Loss = 22.128452
Epoch 310: Loss = 21.716179
Epoch 320: Loss = 21.357349
Epoch 330: Loss = 21.040947
Epoch 340: Loss = 20.646727
Epoch 350: Loss = 20.196730
Epoch 36

In [6]:
# # ------------------------------
# # Evaluate the trained model at select time points and visualize each solution component.
# # ------------------------------
# # Create a fine grid over [0,10]
# t_plot = torch.linspace(0, 10, 200, dtype=torch.float32, device=device)  # shape: (200,)
# # Evaluate the model on this grid. (Ensure correct shape by unsqueezing.)
# y_plot = model(t_plot.unsqueeze(1))  # shape: (200, 5)
# y_plot = y_plot.detach().cpu().numpy()
# t_plot_np = t_plot.detach().cpu().numpy()

# # Create a plot for each component y[0]...y[4]
# plt.figure(figsize=(10, 8))
# for i in range(5):
#     plt.plot(t_plot_np, y_plot[:, i], label=f"y[{i}]")
# plt.xlabel("t")
# plt.ylabel("y")
# plt.title("PINN Solution Components over [0,10]")
# plt.legend()
# plt.grid(True)
# plt.show()

In [7]:
# Save the model's state dictionary to a file
import datetime

# Get the current date in YYYY_MM_DD format
current_date = datetime.date.today().strftime("%Y_%m_%d")

# Create the filename with the date stamp
filename = f"pinn_model_{current_date}.pt"

# Save the model state dictionary
torch.save(model.state_dict(), filename)

print(f"Model saved to {filename}")

Model saved to pinn_model_2025_02_20.pt


In [8]:
# # Recreate the model architecture and move it to the appropriate device
# model_reloaded = PINN().to(device)
# # Load the saved state dictionary
# model_reloaded.load_state_dict(torch.load("pinn_model.pt", map_location=device))
# # Optionally, set the model to evaluation mode
# model_reloaded.eval()